In [80]:
# imports
import pandas as pd

from copy import copy
from openpyxl import load_workbook
from openpyxl.cell import Cell
from openpyxl.formula.translate import Translator
from openpyxl.worksheet import worksheet
from pandas import DataFrame
from sqlalchemy import create_engine

# configuration
sql_dialect = 'mssql'
sql_driver = 'pyodbc'
sql_conn = '@./Goal-Based-Database?'
driver = 'driver=ODBC+Driver+17+for+SQL+Server'

# inputs
sql = 'SELECT Cost, 0.0 FROM CashFlow'
template_name = 'Template.xlsx'
sh_name = 'CashFlow'
start_row = 2
output_name = 'output.xlsx'

# classes
class Config():
    def __init__(self, sql_dialect, sql_driver, sql_conn, driver):
        self.sql_dialect = sql_dialect
        self.sql_driver = sql_driver
        self.sql_conn = sql_conn
        self.driver = driver
        
    def get_engine_conn(self) -> str:
        return self.sql_dialect + '+' + self.sql_driver + '://' + self.sql_conn + self.driver
        
class ReportContext():
    def __init__(self, sql, template_name, sh_name, start_row, output_name):
        self.sql = sql
        self.template_name = template_name
        self.sh_name = sh_name
        self.start_row = start_row
        self.output_name = output_name

class ExcelExport():
    def __init__(self, report_context:ReportContext):
        df = pd.read_sql(report_context.sql, engine)
        wb = load_workbook(filename = report_context.template_name)
        ws = wb[report_context.sh_name]
        ws = self.__get_populated_worksheet(ws, df, report_context.start_row)
        wb.save(report_context.output_name)
        
    def __get_populated_worksheet(self, ws: worksheet, df: DataFrame, start_row: int) -> worksheet:
        cols = df.shape[1]
        rows = df.shape[0]
        for col in range(cols):
            reference = ws.cell(row=start_row, column=col + 1)
            for row in range(rows):
                cell = ws.cell(row=row+start_row+1, column=col+1)
                cell.value = self.__get_cell_value(ws, df, cell, reference)
                cell = self.__get_formatted_cell(cell,reference)
        ws.delete_rows(start_row)
        return ws
    
    def __get_cell_value(self, ws: worksheet, df: DataFrame, cell: Cell, reference: Cell):
        if reference.data_type == 'f':
            coordinate = ws.cell(row=cell.row - 1, column=cell.column).coordinate
            return Translator(reference.value, reference.coordinate).translate_formula(coordinate)
        else:
            df_row = cell.row - reference.row - 1
            return df.values[df_row,cell.column - 1]

    def __get_formatted_cell(self, cell: Cell, reference: Cell) -> Cell:
        if reference.has_style:
            cell.font = copy(reference.font)
            cell.border = copy(reference.border)
            cell.fill = copy(reference.fill)
            cell.number_format = copy(reference.number_format)
            cell.protection = copy(reference.protection)
            cell.alignment = copy(reference.alignment)
        return cell
    
# execute
config = Config(sql_dialect, sql_driver, sql_conn, driver)
engine = create_engine(config.get_engine_conn())

report_context = ReportContext(sql, template_name, sh_name, start_row, output_name)
excel_export = ExcelExport(report_context)
